In [1]:
#Specify directory here
the_folder = '/Users/christopherrauh/Dropbox/text analysis sandbox/programs/competition_replication/'

from sklearn.model_selection import cross_val_score

import pandas as pd
from pathlib import Path

#from mlxtend.classifier import StackingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV


import operator as op
import math

from shutil import copyfile

#from itertools import zip
import numpy as np
import os
import sys
from sklearn.ensemble import RandomForestRegressor# Instantiate model with 1000 decision trees
from sklearn.ensemble import RandomForestClassifier
#from sklearn import model_selection
import copy

import csv


my_file = Path(the_folder+'input/predict_best_15.csv')
data = pd.read_csv(my_file, sep=',')

#function prepares dataset for training and prediction
def in_out(X, which, depo, futuro):

	
	hurz = []

	for ei in which:
		hurz.append(ei)
	hurz.append('isocode')

	#create variables and country list for prediction into the future 
	future_ready = futuro[hurz]
	future_ready = future_ready.dropna(how='any')
	countries_ready = future_ready[['isocode']]
	future_ready = future_ready.drop('isocode', axis=1)

	#create variables and outcomes to train model on past data 
	hurz = []
	for ei in which:
		hurz.append(ei)
	hurz.append(depo)
	X = X[hurz]
	X = X.dropna(how='any')							
	df2 = X[[depo]]
	X = X[which]
	y = df2.values.ravel()

	#scale everything (required for neural network)
	scaler = StandardScaler()
	scaler.fit(X)
	haurein = scaler.transform(X)
	if len(future_ready) > 0:
		hauraus = scaler.transform(future_ready)
	else:
		hauraus = []


	return haurein, y, hauraus, countries_ready, scaler

In [3]:
#provide forecasts for 2014 to 2020
rf = RandomForestRegressor(random_state = 42, criterion='mse')# Train the model on training data

#declaring predictors

#text predictors
text = []
tops = 15
for z in range(0,tops):
    this = 'ste_theta' + str(z)
    text.append(this)
this = 'tokens'
text.append(this)

#set of conflict history predictors
dummies = ['anyviolence_dp', 'armedconf_dp', 'civilwar_dp']
others = ['best', 'past6', 'past12', 'past60', 'past120']
for z in range(1,4):
    this = 'L' + str(z) + 'best'
    others.append(this)
history = dummies + others

#set of all predictors
both = text + history


my_file = Path(the_folder+'/input/predict_best_'+str(tops)+'.csv')



#random forest or lasso
for model in [0]:
    if model == 0:
        modelname = 'rf'
    if model == 1:
        modelname = 'lasso'
    for mine in [both, text, history]:    
        if mine == history:
            namit = 'history'
        if mine == text:
            namit = 'text'
        if mine == both:
            namit = 'both'
        print(namit)
        #predicting these many months ahead
        for f in range(2,8):

            for a in [3]:
                if a == 0:
                    dep = 'goal' + str(f)            
                elif a == 3:
                    dep = 'gold' + str(f)

                #predicting these years
                for y in range(2013, 2021):

                    start_m = 1
                    end_m = 13
                    if y == 2013:
                        start_m = 11
                    elif y == 2020:
                        start_m = 8
                        end_m = 9
                        

                    for m in range(start_m,end_m):
                        
                        
                        find_model = 0
                        if y == 2013 and m == start_m:
                            my_file = Path(the_folder+'input/predict_best_'+str(tops)+'_2013_11.csv')
                            find_model = 1
                        elif y == 2016 and m == 11:
                            my_file = Path(the_folder+'input/predict_best_'+str(tops)+'_2016_11.csv')
                            find_model = 1
                        elif y == 2020 and m == start_m:
                            my_file = Path(the_folder+'input/predict_best_'+str(tops)+'_2020_8.csv')
                            find_model = 1
                        if find_model == 1:
                            data = pd.read_csv(my_file, sep=',')
                            future = data



                            [X, Y, future_out, countries, scalers] = in_out(data, mine, dep, future)

                            if model == 0:
                                tuned_parameters = [{'max_depth': [6,7,8],'n_estimators': [500,650],'min_samples_split': [2],'max_features': ['sqrt'],'min_samples_leaf': [4],}]
                            if model == 1:
                                tuned_parameters = {'alpha':[0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25]}

                            if a != 1:
                                if model == 0:
                                    rf = RandomForestRegressor(random_state = 42, criterion='mse')# Train the model on training data
                                if model == 1:
                                    rf = linear_model.Lasso()
                                clfx = GridSearchCV(rf, tuned_parameters, cv=3)
                            else:
                                rf = RandomForestClassifier(random_state = 42)
                                clfx = GridSearchCV(rf, tuned_parameters, cv=3, scoring='roc_auc')



                            clfx.fit(X, Y)
                            print("Best parameters set found on development set:")
                            print()
                            print(clfx.best_params_)
                            print("Eval criteria :",clfx.best_score_)
                            print()

                            
                            
                            clf = RandomForestRegressor(max_depth=clfx.best_params_['max_depth'],n_estimators =clfx.best_params_['n_estimators'],min_samples_split =clfx.best_params_['min_samples_split'],max_features=clfx.best_params_['max_features'],min_samples_leaf=clfx.best_params_['min_samples_leaf'], random_state=42)
                            
 
                            

                        
                        print(modelname, ' ', namit, ' ', y, ' ', m, ' ', f)
 
                        my_file = Path(the_folder+'input/predict_best_'+str(tops)+'_'+str(y)+'_'+str(m)+'.csv')
                        data = pd.read_csv(my_file, sep=',')
                        #predicting y, m
                        future = data.drop(data[(data.year != y) | (data.month != m)].index)
                        #with data without future years or future months                    
                        data = data.drop(data[(data.year > y)].index)  
 
                        data = data.drop(data[(data.year == y) & (data.month >= m)].index)            
 
                        [X, Y, future_out, countries, scalers] = in_out(data, mine, dep, future)

 
                        clf.fit(X, Y)

                        #predict out of sample
                        if len(future_out) > 0: 
                            if a != 1:
                                res = clf.predict(future_out)
                            else:
                                res = clf.predict_proba(future_out)

                        else:
                            res = []

                        #write out of sample predictions to file
                        if len(res) > 0:
                            cups = countries
                            if a != 1:
                                bla = pd.DataFrame(res[:])
                            else:
                                bla = pd.DataFrame(res[:,1])
                            ei = cups['isocode'].values.tolist()
                            bla = bla.assign(isocode= ei)
                            bla.to_csv(the_folder+'output/'+modelname+'_'+namit+'_topics'+str(tops)+"_m"+str(m)+"_y"+str(y)+"_forward"+str(f)+"_type"+str(a)+".csv")




both


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterio

Best parameters set found on development set:

{'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 650}
Eval criteria : 0.7200506636757936

rf   both   2013   11   2
rf   both   2013   12   2
rf   both   2014   1   2
rf   both   2014   2   2
rf   both   2014   3   2
rf   both   2014   4   2
rf   both   2014   5   2
rf   both   2014   6   2
rf   both   2014   7   2
rf   both   2014   8   2
rf   both   2014   9   2
rf   both   2014   10   2
rf   both   2014   11   2
rf   both   2014   12   2
rf   both   2015   1   2
rf   both   2015   2   2
rf   both   2015   3   2
rf   both   2015   4   2
rf   both   2015   5   2
rf   both   2015   6   2
rf   both   2015   7   2
rf   both   2015   8   2
rf   both   2015   9   2
rf   both   2015   10   2
rf   both   2015   11   2
rf   both   2015   12   2
rf   both   2016   1   2
rf   both   2016   2   2
rf   both   2016   3   2
rf   both   2016   4   2
rf   both   2016   5   2
rf   both   2016   6   2


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterio

Best parameters set found on development set:

{'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 650}
Eval criteria : 0.73374890380096

rf   both   2016   11   2
rf   both   2016   12   2
rf   both   2017   1   2
rf   both   2017   2   2
rf   both   2017   3   2
rf   both   2017   4   2
rf   both   2017   5   2
rf   both   2017   6   2
rf   both   2017   7   2
rf   both   2017   8   2
rf   both   2017   9   2
rf   both   2017   10   2
rf   both   2017   11   2
rf   both   2017   12   2
rf   both   2018   1   2
rf   both   2018   2   2
rf   both   2018   3   2
rf   both   2018   4   2
rf   both   2018   5   2
rf   both   2018   6   2
rf   both   2018   7   2
rf   both   2018   8   2
rf   both   2018   9   2
rf   both   2018   10   2
rf   both   2018   11   2
rf   both   2018   12   2
rf   both   2019   1   2
rf   both   2019   2   2
rf   both   2019   3   2
rf   both   2019   4   2
rf   both   2019   5   2
rf   both   2019   6   2
rf

...

Best parameters set found on development set:

{'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 650}
Eval criteria : 0.6824913187231337

rf   history   2013   11   7
rf   history   2013   12   7
rf   history   2014   1   7
rf   history   2014   2   7
rf   history   2014   3   7
rf   history   2014   4   7
rf   history   2014   5   7
rf   history   2014   6   7
rf   history   2014   7   7
rf   history   2014   8   7
rf   history   2014   9   7
rf   history   2014   10   7
rf   history   2014   11   7
rf   history   2014   12   7
rf   history   2015   1   7
rf   history   2015   2   7
rf   history   2015   3   7
rf   history   2015   4   7
rf   history   2015   5   7
rf   history   2015   6   7
rf   history   2015   7   7
rf   history   2015   8   7
rf   history   2015   9   7
rf   history   2015   10   7
rf   history   2015   11   7
rf   history   2015   12   7
rf   history   2016   1   7
rf   history   2016   2   7
rf   history   2

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterio

Best parameters set found on development set:

{'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 650}
Eval criteria : 0.6824913187231337

rf   history   2016   11   7
rf   history   2016   12   7
rf   history   2017   1   7
rf   history   2017   2   7
rf   history   2017   3   7
rf   history   2017   4   7
rf   history   2017   5   7
rf   history   2017   6   7
rf   history   2017   7   7
rf   history   2017   8   7
rf   history   2017   9   7
rf   history   2017   10   7
rf   history   2017   11   7
rf   history   2017   12   7
rf   history   2018   1   7
rf   history   2018   2   7
rf   history   2018   3   7
rf   history   2018   4   7
rf   history   2018   5   7
rf   history   2018   6   7
rf   history   2018   7   7
rf   history   2018   8   7
rf   history   2018   9   7
rf   history   2018   10   7
rf   history   2018   11   7
rf   history   2018   12   7
rf   history   2019   1   7
rf   history   2019   2   7
rf   history   2

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterio

Best parameters set found on development set:

{'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 650}
Eval criteria : 0.6824913187231337

rf   history   2020   8   7
